# Jaguar Re-Identification

## Score: .873

In [1]:
import os
import math
import random
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd
from PIL import Image
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Sampler

import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cuda


In [2]:
# =============================================================================
# CONFIG — back to what worked (0.883): 224px, 12 epochs, single model, TTA only
# =============================================================================
class CFG:
    data_dir = Path('jaguar-re-id')
    train_csv = data_dir / 'train.csv'
    test_csv = data_dir / 'test.csv'
    train_dir = data_dir / 'train' / 'train'
    test_dir = data_dir / 'test' / 'test'
    
    backbone = 'convnext_base.fb_in22k_ft_in1k'
    image_size = 224
    num_classes = 31
    
    epochs = 12
    batch_size = 32
    lr = 2e-4
    weight_decay = 0.01
    warmup_epochs = 2
    
    arcface_s = 30.0
    arcface_m = 0.5
    label_smoothing = 0.1
    
    samples_per_class = 60
    
    use_tta = True
    use_multiscale_tta = False
    use_qe = False
    use_rerank = False
    
    train_seeds = [42]
    
    num_workers = 0
    mixed_precision = True

In [3]:
# =============================================================================
# DATA
# =============================================================================
train_df = pd.read_csv(CFG.train_csv)
test_df = pd.read_csv(CFG.test_csv)
print(f"Train: {len(train_df)} | Test pairs: {len(test_df)}")

Train: 1895 | Test pairs: 137270


In [4]:
# =============================================================================
# TRANSFORMS
# =============================================================================
def get_train_transforms():
    return A.Compose([
        A.LongestMaxSize(max_size=CFG.image_size),
        A.PadIfNeeded(CFG.image_size, CFG.image_size, border_mode=0),
        A.HorizontalFlip(p=0.5),
        A.Affine(scale=(0.9, 1.1), rotate=(-12, 12), shear=(-8, 8), p=0.5),
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.08, p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])

def get_test_transforms(flip=False, size=None):
    sz = size if size is not None else CFG.image_size
    t = [
        A.LongestMaxSize(max_size=sz),
        A.PadIfNeeded(sz, sz, border_mode=0),
    ]
    if flip:
        t.append(A.HorizontalFlip(p=1.0))
    t.extend([
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])
    return A.Compose(t)

In [5]:
# =============================================================================
# DATASET & SAMPLER
# =============================================================================
class JaguarDataset(Dataset):
    def __init__(self, df, img_dir, transform):
        self.df = df.reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.transform = transform
        unique_ids = sorted(df['ground_truth'].unique())
        self.label_map = {name: i for i, name in enumerate(unique_ids)}
        self.labels = [self.label_map[gt] for gt in df['ground_truth']]
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = np.array(Image.open(self.img_dir / row['filename']).convert('RGB'))
        img = self.transform(image=img)['image']
        return img, torch.tensor(self.labels[idx], dtype=torch.long)


class JaguarTestDataset(Dataset):
    def __init__(self, filenames, img_dir, transform):
        self.filenames = filenames
        self.img_dir = Path(img_dir)
        self.transform = transform
    
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, idx):
        fname = self.filenames[idx]
        img = np.array(Image.open(self.img_dir / fname).convert('RGB'))
        img = self.transform(image=img)['image']
        return img, fname


class BalancedSampler(Sampler):
    def __init__(self, labels, samples_per_class):
        self.labels = labels
        self.samples_per_class = samples_per_class
        self.class_indices = defaultdict(list)
        for idx, label in enumerate(labels):
            self.class_indices[label].append(idx)
        self.num_classes = len(self.class_indices)
    
    def __iter__(self):
        indices = []
        for label in self.class_indices:
            class_idx = self.class_indices[label]
            if len(class_idx) >= self.samples_per_class:
                sampled = random.sample(class_idx, self.samples_per_class)
            else:
                sampled = random.choices(class_idx, k=self.samples_per_class)
            indices.extend(sampled)
        random.shuffle(indices)
        return iter(indices)
    
    def __len__(self):
        return self.num_classes * self.samples_per_class

In [6]:
# =============================================================================
# MODEL
# =============================================================================
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super().__init__()
        self.p = nn.Parameter(torch.ones(1) * p)
        self.eps = eps
    
    def forward(self, x):
        return F.avg_pool2d(x.clamp(min=self.eps).pow(self.p), (x.size(-2), x.size(-1))).pow(1.0 / self.p)


class ArcFaceLoss(nn.Module):
    def __init__(self, in_features, num_classes, s=30.0, m=0.5):
        super().__init__()
        self.s, self.m = s, m
        self.weight = nn.Parameter(torch.FloatTensor(num_classes, in_features))
        nn.init.xavier_uniform_(self.weight)
    
    def forward(self, x, labels):
        cosine = F.linear(F.normalize(x), F.normalize(self.weight))
        theta = torch.acos(cosine.clamp(-1 + 1e-7, 1 - 1e-7))
        target_logits = torch.cos(theta + self.m)
        one_hot = F.one_hot(labels, num_classes=cosine.size(1)).float()
        output = cosine * (1 - one_hot) + target_logits * one_hot
        return output * self.s


class JaguarModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model(CFG.backbone, pretrained=True, num_classes=0)
        self.feat_dim = self.backbone.num_features
        self.gem = GeM()
        self.bn = nn.BatchNorm1d(self.feat_dim)
        self.dropout = nn.Dropout(0.1)
        self.arcface = ArcFaceLoss(self.feat_dim, CFG.num_classes, CFG.arcface_s, CFG.arcface_m)
        print(f"Loaded {CFG.backbone} | Features: {self.feat_dim}")
    
    def extract(self, x):
        features = self.backbone.forward_features(x)
        if features.dim() == 3:
            B, N, C = features.shape
            H = W = int(math.sqrt(N))
            features = features.permute(0, 2, 1).reshape(B, C, H, W)
        emb = self.gem(features).flatten(1)
        emb = self.bn(emb)
        return emb
    
    def forward(self, x, labels=None):
        emb = self.extract(x)
        if labels is not None:
            emb = self.dropout(emb)
            return self.arcface(emb, labels)
        return emb

In [7]:
# =============================================================================
# POST-PROCESSING
# =============================================================================
def query_expansion(emb, top_k=None):
    top_k = top_k if top_k is not None else getattr(CFG, 'qe_top_k', 3)
    print("Applying Query Expansion...")
    sims = emb @ emb.T
    indices = np.argsort(-sims, axis=1)[:, :top_k]
    new_emb = np.zeros_like(emb)
    for i in range(len(emb)):
        new_emb[i] = np.mean(emb[indices[i]], axis=0)
    return new_emb / np.linalg.norm(new_emb, axis=1, keepdims=True)


def k_reciprocal_rerank(prob, k1=20, k2=6, lambda_value=None):
    lambda_value = lambda_value if lambda_value is not None else getattr(CFG, 'rerank_lambda', 0.3)
    print("Applying Re-ranking...")
    q_g_dist = 1 - prob
    original_dist = q_g_dist.copy()
    initial_rank = np.argsort(original_dist, axis=1)
    
    nn_k1 = []
    for i in range(prob.shape[0]):
        forward_k1 = initial_rank[i, :k1+1]
        backward_k1 = initial_rank[forward_k1, :k1+1]
        fi = np.where(backward_k1 == i)[0]
        nn_k1.append(forward_k1[fi])
    
    jaccard_dist = np.zeros_like(original_dist)
    for i in range(prob.shape[0]):
        ind_non_zero = np.where(original_dist[i, :] < 0.6)[0]
        ind_images = [inv for inv in ind_non_zero if len(np.intersect1d(nn_k1[i], nn_k1[inv])) > 0]
        for j in ind_images:
            intersection = len(np.intersect1d(nn_k1[i], nn_k1[j]))
            union = len(np.union1d(nn_k1[i], nn_k1[j]))
            jaccard_dist[i, j] = 1 - intersection / union
    
    return 1 - (jaccard_dist * lambda_value + original_dist * (1 - lambda_value))

In [8]:
# =============================================================================
# TRAINING (two seeds by default for ensemble)
# =============================================================================
train_seeds = getattr(CFG, 'train_seeds', [42])
for run_idx, seed in enumerate(train_seeds):
    seed_everything(seed)
    ckpt_name = 'best_model.pth' if run_idx == 0 else 'best_model_s2.pth'
    train_dataset = JaguarDataset(train_df, CFG.train_dir, get_train_transforms())
    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.batch_size,
        sampler=BalancedSampler(train_dataset.labels, CFG.samples_per_class),
        num_workers=CFG.num_workers,
        pin_memory=True
    )
    model = JaguarModel().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr=CFG.lr, epochs=CFG.epochs, steps_per_epoch=len(train_loader),
        pct_start=CFG.warmup_epochs/CFG.epochs
    )
    scaler = torch.amp.GradScaler('cuda')
    criterion = nn.CrossEntropyLoss(label_smoothing=CFG.label_smoothing)
    best_loss = float('inf')
    for epoch in range(CFG.epochs):
        model.train()
        total_loss = 0
        pbar = tqdm(train_loader, desc=f'Seed {seed} Epoch {epoch+1}/{CFG.epochs}')
        for imgs, labels in pbar:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            with torch.amp.autocast('cuda'):
                logits = model(imgs, labels)
                loss = criterion(logits, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            total_loss += loss.item()
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | LR: {scheduler.get_last_lr()[0]:.2e}")
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(model.state_dict(), ckpt_name)
    print(f"Run {run_idx+1} (seed {seed}) complete | Best loss: {best_loss:.4f} | Saved {ckpt_name}")

Loaded convnext_base.fb_in22k_ft_in1k | Features: 1024


Seed 42 Epoch 1/12:   0%|          | 0/59 [00:00<?, ?it/s]

C:\Users\ol1v3_7dwns5u\AppData\Roaming\Python\Python311\site-packages\torch\optim\lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 1 | Loss: 15.5181 | LR: 1.05e-04


Seed 42 Epoch 2/12:   0%|          | 0/59 [00:00<?, ?it/s]

...

Epoch 10 | Loss: 0.7951 | LR: 1.88e-05


Seed 42 Epoch 11/12:   0%|          | 0/59 [00:00<?, ?it/s]

Epoch 11 | Loss: 0.7616 | LR: 4.73e-06


Seed 42 Epoch 12/12:   0%|          | 0/59 [00:00<?, ?it/s]

Epoch 12 | Loss: 0.7639 | LR: 2.22e-09
Run 1 (seed 42) complete | Best loss: 0.7616 | Saved best_model.pth


In [9]:
# =============================================================================
# INFERENCE
# =============================================================================
unique_images = sorted(set(test_df['query_image']) | set(test_df['gallery_image']))
print(f"Extracting embeddings for {len(unique_images)} images...")

def extract_embeddings(transform, m):
    loader = DataLoader(
        JaguarTestDataset(unique_images, CFG.test_dir, transform),
        batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers
    )
    feats, names = [], []
    with torch.no_grad():
        for imgs, fnames in tqdm(loader, leave=False):
            emb = m(imgs.to(device))
            feats.append(F.normalize(emb, dim=1).cpu())
            names.extend(fnames)
    return torch.cat(feats, dim=0), names

def get_embeddings(m, size=None):
    sz = size if size is not None else CFG.image_size
    e1, names = extract_embeddings(get_test_transforms(flip=False, size=sz), m)
    if CFG.use_tta:
        e2, _ = extract_embeddings(get_test_transforms(flip=True, size=sz), m)
        e = F.normalize((e1 + e2) / 2, dim=1)
    else:
        e = e1
    return e.numpy(), names

model.load_state_dict(torch.load('best_model.pth'))
model.eval()

if getattr(CFG, 'use_multiscale_tta', False) and getattr(CFG, 'multiscale_sizes', None):
    embs = []
    for sz in CFG.multiscale_sizes:
        e, names = get_embeddings(model, size=sz)
        embs.append(e)
    emb = np.stack(embs).mean(axis=0)
    emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)
else:
    emb, names = get_embeddings(model)

if Path('best_model_s2.pth').exists():
    print("Ensemble: loading second model...")
    model2 = JaguarModel().to(device)
    model2.load_state_dict(torch.load('best_model_s2.pth'))
    model2.eval()
    if getattr(CFG, 'use_multiscale_tta', False) and getattr(CFG, 'multiscale_sizes', None):
        embs2 = [get_embeddings(model2, size=sz)[0] for sz in CFG.multiscale_sizes]
        emb2 = np.stack(embs2).mean(axis=0)
        emb2 = emb2 / np.linalg.norm(emb2, axis=1, keepdims=True)
    else:
        emb2, _ = get_embeddings(model2)
    emb = (emb + emb2) / 2
    emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)

img_map = {n: i for i, n in enumerate(names)}

Extracting embeddings for 371 images...


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

Ensemble: loading second model...
Loaded convnext_base.fb_in22k_ft_in1k | Features: 1024


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [10]:
# =============================================================================
# POST-PROCESSING & SUBMISSION
# =============================================================================
if CFG.use_qe:
    emb = query_expansion(emb)

sim_matrix = emb @ emb.T

if CFG.use_rerank:
    sim_matrix = k_reciprocal_rerank(sim_matrix)

print("Computing similarities...")
preds = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    s = sim_matrix[img_map[row['query_image']], img_map[row['gallery_image']]]
    preds.append(float(np.clip(s, 0, 1)))

submission = pd.DataFrame({'row_id': test_df['row_id'], 'similarity': preds})
submission.to_csv('submission.csv', index=False)
print(f"Saved submission.csv | Mean sim: {np.mean(preds):.4f}")

Computing similarities...


  0%|          | 0/137270 [00:00<?, ?it/s]

Saved submission.csv | Mean sim: 0.0580
